In [1]:
import numpy as np
import math
import pandas as pd

In [2]:
def binary_tree_bsm(S, X, T, vol, r, b, N, option_type="call"):
    option_type = str.lower(option_type)
    if option_type not in ["call", "put"]:
        raise ValueError("Option type must be call or put")

    
    dt = T/N
    u = np.exp(vol*np.sqrt(dt))
    d = np.exp(-vol*np.sqrt(dt))
    p_u = (np.exp(b*dt)-d)/(u-d)
    p_d = 1.0-p_u

    path_probabilities = []
    path_prices = []
    num_path_occurrences = []
    
    for i in range(N+1):
        path_probabilities.append(p_u**(i)*p_d**(N-i))
        num_path_occurrences.append(math.comb(N, i))
        path_prices.append(S * u**(i) * d**(N-i))
    
    for i in range(N+1):
        if option_type == "call":
            path_prices[i] = max(0, path_prices[i] - X)
        else:
            path_prices[i] = max(0, X - path_prices[i])
    
    option_value = 0.0
    for i in range(N+1):
        option_value += path_prices[i] * (path_probabilities[i] * num_path_occurrences[i])

    return np.exp(-r*T) * option_value


In [4]:
option_type = "call"
S = 75
X = 77
days = 90
dpy = 360
T = days/dpy
vol = 0.15
r = 0.04
div = 0.01
b = r - div # rfr - dividend
value = binary_tree_bsm(S, X, T, vol, r, b, N=1000, option_type=option_type)
value

np.float64(1.619823540393225)

In [7]:
data_12_1 = pd.read_csv("testfiles/data/test12_1.csv").dropna()
test_12_1 = pd.read_csv("testfiles/data/testout12_1.csv")
test_12_1 = test_12_1.set_index("ID")

error_epsilon = 1e-5
for index, row in data_12_1.iterrows():
    id = row["ID"]
    option_type = row["Option Type"]
    S = row["Underlying"]
    X = row["Strike"]
    days = row["DaysToMaturity"]
    dpy = row["DayPerYear"]
    T = days/dpy
    vol = row["ImpliedVol"]
    r = row["RiskFreeRate"]
    div = row["DividendRate"]
    b = r - div # rfr - dividend
    # b = 0
    
    value = binary_tree_bsm(S, X, T, vol, r, b, N=10_000, option_type=option_type)
    t_value, t_delta, t_gamma, t_vega, t_rho, t_theta = test_12_1.loc[id]

    print(value, t_value)

KeyboardInterrupt: 